# Time Series Clustering with `cluster()`

Accelerate investment optimization using typical periods (clustering).

This notebook demonstrates:

- **Typical periods**: Cluster similar time segments (e.g., days) and solve only representative ones
- **Weighted costs**: Automatically weight operational costs by cluster occurrence
- **Two-stage workflow**: Fast sizing with clustering, accurate dispatch at full resolution

!!! note "Requirements"
    This notebook requires the `tsam` package: `pip install tsam`

In [ ]:
import timeit

import pandas as pd
import xarray as xr

import flixopt as fx

fx.CONFIG.notebook()

## Create the FlowSystem

We use a district heating system with real-world time series data (one month at 15-min resolution):

In [ ]:
from data.generate_example_systems import create_district_heating_system

flow_system = create_district_heating_system()
flow_system.connect_and_transform()

timesteps = flow_system.timesteps
print(f'Loaded FlowSystem: {len(timesteps)} timesteps ({len(timesteps) / 24:.0f} days at hourly resolution)')
print(f'Components: {list(flow_system.components.keys())}')

In [ ]:
# Visualize input data
input_ds = xr.Dataset(
    {
        'Heat Demand': flow_system.components['HeatDemand'].inputs[0].fixed_relative_profile,
        'Electricity Price': flow_system.components['GridBuy'].outputs[0].effects_per_flow_hour['costs'],
    }
)
input_ds.fxplot.line(facet_row='variable', title='One Month of Input Data')

## Method 1: Full Optimization (Baseline)

First, solve the complete problem with all 2976 timesteps:

In [ ]:
solver = fx.solvers.HighsSolver(mip_gap=0.01)

start = timeit.default_timer()
fs_full = flow_system.copy()
fs_full.optimize(solver)
time_full = timeit.default_timer() - start

print(f'Full optimization: {time_full:.1f} seconds')
print(f'Total cost: {fs_full.solution["costs"].item():,.0f} €')
print('\nOptimized sizes:')
for name, size in fs_full.statistics.sizes.items():
    print(f'  {name}: {float(size.item()):.1f}')

## Method 2: Clustering with `cluster()`

The `cluster()` method:

1. **Clusters similar days** using the TSAM (Time Series Aggregation Module) package
2. **Reduces timesteps** to only typical periods (e.g., 8 typical days = 768 timesteps)
3. **Weights costs** by how many original days each typical day represents
4. **Handles storage** with configurable behavior via `storage_mode`

!!! warning "Peak Forcing"
    Always use `time_series_for_high_peaks` to ensure extreme demand days are captured.
    Without this, clustering may miss peak periods, causing undersized components.

In [ ]:
start = timeit.default_timer()

# IMPORTANT: Force inclusion of peak demand periods!
peak_series = ['HeatDemand(Q_th)|fixed_relative_profile']

# Create reduced FlowSystem with 8 typical days
fs_clustered = flow_system.transform.cluster(
    n_clusters=8,  # 8 typical days
    cluster_duration='1D',  # Daily clustering
    time_series_for_high_peaks=peak_series,  # Capture peak demand day
    random_state=42,  # Reproducible results
)

time_clustering = timeit.default_timer() - start

print(
    f'Clustering: {len(flow_system.timesteps)} → {len(fs_clustered.timesteps) * len(fs_clustered.clusters)} timesteps'
)
print(f'  Clusters: {len(fs_clustered.clusters)}')
print(f'  Time: {time_clustering:.2f}s')

In [ ]:
# Optimize the reduced system
start = timeit.default_timer()
fs_clustered.optimize(solver)
time_clustered = timeit.default_timer() - start

print(f'Clustered optimization: {time_clustered:.1f} seconds')
print(f'Total cost: {fs_clustered.solution["costs"].item():,.0f} €')
print(f'\nSpeedup vs full: {time_full / (time_clustering + time_clustered):.1f}x')
print('\nOptimized sizes:')
for name, size in fs_clustered.statistics.sizes.items():
    print(f'  {name}: {float(size.item()):.1f}')

## Understanding the Clustering

The clustering algorithm groups similar days together. Access all metadata via `fs.clustering`:

In [ ]:
# Access clustering metadata directly
clustering = fs_clustered.clustering
clustering

In [ ]:
# Key properties
print(f'Clusters: {clustering.n_clusters}')
print(f'Original segments (days): {clustering.n_original_clusters}')
print(f'Timesteps per cluster: {clustering.timesteps_per_cluster}')
print(f'\nCluster occurrences: {clustering.occurrences.values}')
print(f'Cluster order: {clustering.cluster_order.values}')

In [ ]:
# Quality metrics - how well do the clusters represent the original data?
# Lower RMSE/MAE = better representation
clustering.metrics.to_dataframe().style.format('{:.3f}')

In [ ]:
# Visual comparison: original vs clustered time series
clustering.plot.compare()

## Advanced Clustering Options

The `cluster()` method exposes many parameters for fine-tuning:

In [ ]:
# Try different clustering algorithms
fs_hierarchical = flow_system.transform.cluster(
    n_clusters=8,
    cluster_duration='1D',
    cluster_method='hierarchical',  # Alternative: 'k_means' (default), 'k_medoids', 'averaging'
    random_state=42,
)

# Compare cluster assignments between algorithms
print('k_means clusters:    ', fs_clustered.clustering.cluster_order.values)
print('hierarchical clusters:', fs_hierarchical.clustering.cluster_order.values)

In [ ]:
# Compare RMSE between algorithms
print('Quality comparison (RMSE for HeatDemand):')
print(
    f'  k_means:      {float(fs_clustered.clustering.metrics["RMSE"].sel(time_series="HeatDemand(Q_th)|fixed_relative_profile")):.4f}'
)
print(
    f'  hierarchical: {float(fs_hierarchical.clustering.metrics["RMSE"].sel(time_series="HeatDemand(Q_th)|fixed_relative_profile")):.4f}'
)

In [ ]:
# Visualize cluster structure with heatmap
clustering.plot.heatmap()

## Method 3: Two-Stage Workflow (Recommended)

The recommended approach for investment optimization:

1. **Stage 1**: Fast sizing with `cluster()` 
2. **Stage 2**: Fix sizes (with safety margin) and dispatch at full resolution

!!! tip "Safety Margin"
    Typical periods aggregate similar days, so individual days may have higher demand 
    than the typical day. Adding a 5-10% margin ensures feasibility.

In [ ]:
# Stage 1 already done above
print('Stage 1: Sizing with typical periods')
print(f'  Time: {time_clustering + time_clustered:.1f} seconds')
print(f'  Cost estimate: {fs_clustered.solution["costs"].item():,.0f} €')

# Apply safety margin to sizes
SAFETY_MARGIN = 1.05  # 5% buffer
sizes_with_margin = {name: float(size.item()) * SAFETY_MARGIN for name, size in fs_clustered.statistics.sizes.items()}
print(f'\nSizes with {(SAFETY_MARGIN - 1) * 100:.0f}% safety margin:')
for name, size in sizes_with_margin.items():
    original = fs_clustered.statistics.sizes[name].item()
    print(f'  {name}: {original:.1f} → {size:.1f}')

In [ ]:
# Stage 2: Fix sizes and optimize at full resolution
print('Stage 2: Dispatch at full resolution')
start = timeit.default_timer()

fs_dispatch = flow_system.transform.fix_sizes(sizes_with_margin)
fs_dispatch.optimize(solver)

time_dispatch = timeit.default_timer() - start
print(f'  Time: {time_dispatch:.1f} seconds')
print(f'  Actual cost: {fs_dispatch.solution["costs"].item():,.0f} €')

# Total comparison
total_two_stage = time_clustering + time_clustered + time_dispatch
print(f'\nTotal two-stage time: {total_two_stage:.1f} seconds')
print(f'Speedup vs full: {time_full / total_two_stage:.1f}x')

## Compare Results

In [ ]:
results = {
    'Full (baseline)': {
        'Time [s]': time_full,
        'Cost [€]': fs_full.solution['costs'].item(),
        'CHP': fs_full.statistics.sizes['CHP(Q_th)'].item(),
        'Boiler': fs_full.statistics.sizes['Boiler(Q_th)'].item(),
        'Storage': fs_full.statistics.sizes['Storage'].item(),
    },
    'Clustered (8 days)': {
        'Time [s]': time_clustering + time_clustered,
        'Cost [€]': fs_clustered.solution['costs'].item(),
        'CHP': fs_clustered.statistics.sizes['CHP(Q_th)'].item(),
        'Boiler': fs_clustered.statistics.sizes['Boiler(Q_th)'].item(),
        'Storage': fs_clustered.statistics.sizes['Storage'].item(),
    },
    'Two-Stage': {
        'Time [s]': total_two_stage,
        'Cost [€]': fs_dispatch.solution['costs'].item(),
        'CHP': sizes_with_margin['CHP(Q_th)'],
        'Boiler': sizes_with_margin['Boiler(Q_th)'],
        'Storage': sizes_with_margin['Storage'],
    },
}

comparison = pd.DataFrame(results).T
baseline_cost = comparison.loc['Full (baseline)', 'Cost [€]']
baseline_time = comparison.loc['Full (baseline)', 'Time [s]']
comparison['Cost Gap [%]'] = ((comparison['Cost [€]'] - baseline_cost) / abs(baseline_cost) * 100).round(2)
comparison['Speedup'] = (baseline_time / comparison['Time [s]']).round(1)

comparison.style.format(
    {
        'Time [s]': '{:.1f}',
        'Cost [€]': '{:,.0f}',
        'CHP': '{:.1f}',
        'Boiler': '{:.1f}',
        'Storage': '{:.0f}',
        'Cost Gap [%]': '{:.2f}',
        'Speedup': '{:.1f}x',
    }
)

## Expand Solution to Full Resolution

Use `expand_solution()` to map the clustered solution back to all original timesteps.
This repeats the typical period values for all days belonging to that cluster:

In [ ]:
# Expand the clustered solution to full resolution
fs_expanded = fs_clustered.transform.expand_solution()

print(f'Expanded: {len(fs_clustered.timesteps)} → {len(fs_expanded.timesteps)} timesteps')
print(f'Cost: {fs_expanded.solution["costs"].item():,.0f} €')

In [ ]:
# Compare heat production: Full vs Expanded
heat_flows = ['CHP(Q_th)|flow_rate', 'Boiler(Q_th)|flow_rate']

# Create comparison dataset
comparison_ds = xr.Dataset(
    {
        name.replace('|flow_rate', ''): xr.concat(
            [fs_full.solution[name], fs_expanded.solution[name]], dim=pd.Index(['Full', 'Expanded'], name='method')
        )
        for name in heat_flows
    }
)

comparison_ds.fxplot.line(facet_col='variable', color='method', title='Heat Production Comparison')

## Visualize Clustered Heat Balance

In [ ]:
fs_clustered.statistics.plot.storage('Storage')

In [ ]:
fs_expanded.statistics.plot.storage('Storage').data.to_dataframe()

## API Reference

### `transform.cluster()` Parameters

| Parameter | Type | Default | Description |
|-----------|------|---------|-------------|
| `n_clusters` | `int` | - | Number of typical periods (e.g., 8 typical days) |
| `cluster_duration` | `str \| float` | - | Duration per cluster ('1D', '24h') or hours |
| `weights` | `dict[str, float]` | None | Optional weights for time series in clustering |
| `time_series_for_high_peaks` | `list[str]` | None | **Essential**: Force inclusion of peak periods |
| `time_series_for_low_peaks` | `list[str]` | None | Force inclusion of minimum periods |
| `cluster_method` | `str` | 'k_means' | Algorithm: 'k_means', 'hierarchical', 'k_medoids', 'k_maxoids', 'averaging' |
| `representation_method` | `str` | 'meanRepresentation' | 'meanRepresentation', 'medoidRepresentation', 'distributionAndMinMaxRepresentation' |
| `extreme_period_method` | `str` | 'new_cluster_center' | How peaks are integrated: 'None', 'append', 'new_cluster_center', 'replace_cluster_center' |
| `rescale_cluster_periods` | `bool` | True | Rescale clusters to match original means |
| `random_state` | `int` | None | Random seed for reproducibility |
| `predef_cluster_order` | `array` | None | Manual cluster assignments |
| `**tsam_kwargs` | - | - | Additional tsam parameters |

### Clustering Object Properties

After clustering, access metadata via `fs.clustering`:

| Property | Description |
|----------|-------------|
| `n_clusters` | Number of clusters |
| `n_original_clusters` | Number of original time segments (e.g., 365 days) |
| `timesteps_per_cluster` | Timesteps in each cluster (e.g., 24 for daily) |
| `cluster_order` | xr.DataArray mapping original segment → cluster ID |
| `occurrences` | How many original segments each cluster represents |
| `metrics` | xr.Dataset with RMSE, MAE per time series |
| `plot.compare()` | Compare original vs clustered time series |
| `plot.heatmap()` | Visualize cluster structure |

### Storage Behavior

Each `Storage` component has a `cluster_mode` parameter:

| Mode | Description |
|------|-------------|
| `'intercluster_cyclic'` | Links storage across clusters + yearly cyclic **(default)** |
| `'intercluster'` | Links storage across clusters, free start/end |
| `'cyclic'` | Each cluster is independent but cyclic (start = end) |
| `'independent'` | Each cluster is independent, free start/end |

For a detailed comparison of storage modes, see [08c2-clustering-storage-modes](08c2-clustering-storage-modes.ipynb).

## Summary

You learned how to:

- Use **`cluster()`** to reduce time series into typical periods
- Apply **peak forcing** to capture extreme demand days
- Use **two-stage optimization** for fast yet accurate investment decisions
- **Expand solutions** back to full resolution with `expand_solution()`
- Access **clustering metadata** via `fs.clustering` (metrics, cluster_order, occurrences)
- Use **advanced options** like different algorithms and reproducible random states

### Key Takeaways

1. **Always use peak forcing** (`time_series_for_high_peaks`) for demand time series
2. **Add safety margin** (5-10%) when fixing sizes from clustering
3. **Two-stage is recommended**: clustering for sizing, full resolution for dispatch
4. **Storage handling** is configurable via `cluster_mode`
5. **Use `random_state`** for reproducible results
6. **Check metrics** to evaluate clustering quality

### Next Steps

- **[08c2-clustering-storage-modes](08c2-clustering-storage-modes.ipynb)**: Compare storage modes using a seasonal storage system
- **[08d-clustering-multiperiod](08d-clustering-multiperiod.ipynb)**: Clustering with multiple periods and scenarios